In [1]:
import json
import os
from pyquaternion import Quaternion
import pandas as pd
import pickle
import numpy as np

In [2]:
EXPERIMENT = False

In [3]:
if EXPERIMENT:
    suffix = '_experiment'
else:
    suffix = ''

In [4]:
if 'NUSCENE_DATA' in os.environ:
    base_dir = os.environ['NUSCENE_DATA']
else:
    base_dir = '/work/apperception/data/nuScenes/full-dataset-v1.0/Trainval/'

In [5]:
base_dir

'/Users/chanwutk/Documents/apperception/data/v1.0-mini'

In [6]:
for d in os.listdir(base_dir):
    if d.startswith('v1.0-'):
        folder = d
        break
folder

'v1.0-mini'

In [7]:
# try:
#     os.remove(os.path.join(base_dir, f'annotation{suffix}.csv'))
# except FileNotFoundError:
#     pass

# try:
#     os.remove(os.path.join(base_dir, f'annotation{suffix}.pickle'))
# except FileNotFoundError:
#     pass

# try:
#     os.remove(os.path.join(base_dir, f'sample_data{suffix}.csv'))
# except FileNotFoundError:
#     pass

# try:
#     os.remove(os.path.join(base_dir, f'sample_data{suffix}.pickle'))
# except FileNotFoundError:
#     pass

In [8]:
with open(os.path.join(base_dir, folder, 'calibrated_sensor.json')) as f:
    calibrated_sensor_json = json.load(f)

with open(os.path.join(base_dir, folder, 'category.json')) as f:
    category_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample.json')) as f:
    sample_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample_data.json')) as f:
    sample_data_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample_annotation.json')) as f:
    sample_annotation_json = json.load(f)

with open(os.path.join(base_dir, folder, 'instance.json')) as f:
    instance_json = json.load(f)

with open(os.path.join(base_dir, folder, 'scene.json')) as f:
    scene_json = json.load(f)

with open(os.path.join(base_dir, folder, 'ego_pose.json')) as f:
    ego_pose_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sensor.json')) as f:
    sensor_json = json.load(f)

In [9]:
files = os.listdir(os.path.join(base_dir, 'experiment_data'))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/chanwutk/Documents/apperception/data/v1.0-mini/experiment_data'

In [10]:
import math
from typing import List

def normalizeAngle(angle) -> float:
    while angle > math.pi:
        angle -= math.tau
    while angle < -math.pi:
        angle += math.tau
    assert -math.pi <= angle <= math.pi
    return angle

def get_heading(rotation):
    yaw = rotation.yaw_pitch_roll[0]
    return normalizeAngle(yaw)

def get_heading_from_north(rotation):
    yaw = rotation.yaw_pitch_roll[0]
    return normalizeAngle(yaw - (math.pi / 2))

rot = Quaternion(axis=[1, 0, 0], angle=np.pi / 2)
def get_camera_heading(rotation):
    return -get_heading(rot.rotate(rotation)) + math.pi / 2
    
def get_camera_position(
    camera_translation: List[float],
    ego_translation: List[float],
    ego_rotation: List[float]
) -> np.ndarray:
    rotated_offset = Quaternion(ego_rotation) \
        .rotate(np.array(camera_translation))
    return np.array(ego_translation) + rotated_offset

def get_camera_rotation(
    camera_rotation: List[float],
    ego_rotation: List[float]
) -> np.ndarray:
    return (Quaternion(camera_rotation) * Quaternion(ego_rotation)).q

In [ ]:
files_set = set(files)

In [11]:
if EXPERIMENT:
    sample_data_filter = [s for s in sample_data_json if s['filename'].split('/')[2] in files_set]
else:
    sample_data_filter = sample_data_json
# sample_data_filter = [s for s in sample_data_json if 'CAM_FRONT/' in s['filename']]
sample_data_filter = [s for s in sample_data_filter if s['fileformat'] == 'jpg']

In [12]:
sample_data_filter[0]

{'token': '512015c209c1490f906982c3b182c2a8',
 'sample_token': '39586f9d59004284a7114a68825e8eec',
 'ego_pose_token': '512015c209c1490f906982c3b182c2a8',
 'calibrated_sensor_token': '1d31c729b073425e8e0202c5c6e66ee1',
 'timestamp': 1532402927762460,
 'fileformat': 'jpg',
 'is_key_frame': False,
 'height': 900,
 'width': 1600,
 'filename': 'sweeps/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402927762460.jpg',
 'prev': '68e8e98cf7b0487baa139df808641db7',
 'next': 'e9df123af5b747adbd9ddf6454e1abdc'}

In [13]:
sample_tokens = set([s['sample_token'] for s in sample_data_filter])

In [14]:
ego_pose_tokens = set([s['ego_pose_token'] for s in sample_data_filter])

In [15]:
calibrated_sensor_tokens = set([s['calibrated_sensor_token'] for s in sample_data_filter])

In [16]:
sample_filter = [
    {
        'sample_token': s['token'],
        'scene_token': s['scene_token']
    }
    for s in sample_json
    if s['token'] in sample_tokens
]
len(sample_filter)

404

In [17]:
scene_tokens = set([s['scene_token'] for s in sample_filter])

In [18]:
calibrated_sensor_filter = [
    {
        'calibrated_sensor_token': c['token'],
        'camera_translation': c['translation'],
        'camera_rotation': c['rotation'],
        'camera_intrinsic': c['camera_intrinsic']
    }
    for c in calibrated_sensor_json
    if c['token'] in calibrated_sensor_tokens
]
len(calibrated_sensor_filter)

60

In [19]:
ego_pose_filter = [
    {
        'ego_pose_token': e['token'],
        'ego_translation': e['translation'],
        'ego_rotation': e['rotation']
    }
    for e in ego_pose_json
    if e['token'] in ego_pose_tokens
]
len(ego_pose_filter)

14008

In [20]:
scene_filter = [
    {
        'scene_token': s['token'],
        'scene_name': s['name'],
    }
    for s in scene_json
    if s['token'] in scene_tokens
]
len(scene_filter)

10

In [21]:
sample_map = {
    s['sample_token']: s
    for s in sample_filter
}
calibrated_sensor_map = {
    c['calibrated_sensor_token']: c
    for c in calibrated_sensor_filter
}
ego_pose_map = {
    e['ego_pose_token']: e
    for e in ego_pose_filter
}
scene_map = {
    s['scene_token']: s
    for s in scene_filter
}

def s_map(s):
    sample = sample_map[s['sample_token']]
    calibrated_sensor = calibrated_sensor_map[s['calibrated_sensor_token']]
    ego_pose = ego_pose_map[s['ego_pose_token']]
    ego_heading = get_heading_from_north(Quaternion(ego_pose['ego_rotation']))
    camera_heading = get_camera_heading(Quaternion(calibrated_sensor['camera_rotation']))
    ret = {
        **s,
        **sample,
        **calibrated_sensor,
        **ego_pose,
        **scene_map[sample['scene_token']],
        'ego_heading': ego_heading * 180 / math.pi,
        'camera_heading': normalizeAngle(camera_heading + ego_heading) * 180 / math.pi,
        'camera_translation': get_camera_position(
            calibrated_sensor['camera_translation'],
            ego_pose['ego_translation'],
            ego_pose['ego_rotation'],
        ),
        'camera_rotation': get_camera_rotation(
            calibrated_sensor['camera_rotation'],
            ego_pose['ego_rotation'],
        ),
    }
    del ret['ego_pose_token']
    del ret['calibrated_sensor_token']
    del ret['fileformat']
    del ret['height']
    del ret['width']
    del ret['prev']
    del ret['next']
    del ret['scene_token']
    return ret

sample_data_res = [*map(s_map, sample_data_filter)]

len(sample_data_res)

14008

In [22]:
sample_annotation_filter = [
    sa
    for sa in sample_annotation_json
    if sa['sample_token'] in sample_tokens
]
len(sample_annotation_filter)

18538

In [23]:
instance_tokens = set([
    sa['instance_token']
    for sa in sample_annotation_filter
])

instance_filter = [
    {
        'instance_token': i['token'],
        'category_token': i['category_token']
    }
    for i in instance_json
    if i['token'] in instance_tokens
]
len(instance_filter)

911

In [24]:
category_tokens = set([
    i['category_token']
    for i in instance_filter
])

category_filter = [
    {
        'category_token': c['token'],
        'category': c['name']
    }
    for c in category_json
    if c['token'] in category_tokens
]
len(category_filter)

18

In [25]:
instance_map = {
    i['instance_token']: i
    for i in instance_filter
}
category_map = {
    c['category_token']: c
    for c in category_filter
}

def sa_map(sa):
    instance = instance_map[sa['instance_token']]
    ret = {
        **sa,
        **instance,
        **category_map[instance['category_token']],
        'heading': (get_heading_from_north(Quaternion(sa['rotation']))) * 180 / math.pi
    }
    
    del ret['visibility_token']
    del ret['attribute_tokens']
    del ret['prev']
    del ret['next']
    del ret['num_lidar_pts']
    del ret['num_radar_pts']
    del ret['category_token']
    
    return ret

sample_annotation_res = [*map(sa_map, sample_annotation_filter)]
len(sample_annotation_res)

18538

In [26]:
sample_data_final = pd.DataFrame(sample_data_res)

In [27]:
sample_annotation_final = pd.DataFrame(sample_annotation_res)

In [28]:
df_sample_data_keyframe = sample_data_final[sample_data_final["is_key_frame"]][
    ["token", "sample_token"]
]
sample_annotation_final = sample_annotation_final.set_index("sample_token").join(
    df_sample_data_keyframe.set_index("sample_token"), on="sample_token", rsuffix="_sample_data"
)

In [29]:
sample_data_final = sample_data_final.sort_values(by=['scene_name', 'timestamp'])

In [30]:
sample_data_final["frame_order"] = 0
scene = ""
i = 1
for index, row in sample_data_final.iterrows():
    if row['scene_name'] != scene:
        scene = row['scene_name']
        sample_data_final.loc[index, "frame_order"] = 1
        i = 2
    else:
        sample_data_final.loc[index, "frame_order"] = i
        i += 1


In [31]:
print(len(sample_data_final))

14008


In [32]:
print(len(sample_annotation_final))

111228


In [33]:
with open(os.path.join(base_dir, f"sample_data{suffix}.pickle"), "wb") as f:
    pickle.dump(sample_data_final, f)
with open(os.path.join(base_dir, f"annotation{suffix}.pickle"), "wb") as f:
    pickle.dump(sample_annotation_final, f)

In [34]:
sample_data_final.to_csv(os.path.join(base_dir, f"sample_data{suffix}.csv"), index=False)

In [35]:
sample_annotation_final.to_csv(os.path.join(base_dir, f"annotation{suffix}.csv"), index=False)

In [ ]:
calibrated_sensor = [
    {
        **c,
        **[s for s in sensor_json if s['token'] == c['sensor_token']][0]
    }
    for c in calibrated_sensor_json
]
calibrated_sensor = [
    c
    for c in calibrated_sensor
    if c['modality'] == 'camera'
]
rot = Quaternion(axis=[1, 0, 0], angle=np.pi / 2)
[
    normalizeAngle(get_heading(rot.rotate(Quaternion(c['rotation'])))) * 180 / math.pi
    for c in calibrated_sensor
    if c['channel'] == "CAM_FRONT"
]

In [ ]:
sample_data_final